In [1]:
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.core import Dropout
def res_net():
    input = Input(shape=[100, 100, 3])
    nub_class = 7
    base_model = VGG16(include_top=False,weights='imagenet',input_tensor=input)
    x = base_model.output
    x = Flatten()(x)
    x = Dropout(0.4)(x)
    x = Dense(1024,activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(256,activation='relu')(x)
    x = Dense(nub_class,activation='softmax')(x)
    model = Model(input, x)
    
    return model

F:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model = res_net()

In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 49, 49, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 49, 49, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 49, 49, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [4]:
import os
import numpy as np
from PIL import Image
filelist = []
filenamelist = []
file_dir = 'F:/ingnew3/2018.8.demo/图像人脸定位/experiment_emotion/'
for root, dirs, files in os.walk(file_dir):
    filelist.append(files)
filenamelist=filelist[1:8]
imagelist=[]
for i in range(len(filenamelist)):
    for j in range(len(filenamelist[i])):
        image = Image.open(file_dir+str(i)+'/'+filenamelist[i][j])
        reimg = image.resize((100,100))
        im_array = np.array(reimg)
        imagelist.append(im_array)
label_list=[]
tmp=0
for i in range(len(filenamelist)):
    for j in range(len(filenamelist[i])):
        label_list.append(tmp)
    tmp=tmp+1
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imagelist, label_list, test_size=0.3, random_state=1)
x_train_array = np.array(X_train,dtype=np.float32)
x_test_array = np.array(X_test,dtype=np.float32)
from keras.utils import np_utils
num_classes = 7
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

F:\anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
len(x_train_array)

984

In [6]:
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.applications.xception import preprocess_input
sgd = SGD(lr=0.001, momentum=0.9, nesterov=True)
adam = Adam(lr=0.00001)
rmsprop = RMSprop(lr=0.0005)
model.compile(optimizer=adam, loss="categorical_crossentropy", metrics=['accuracy'])
train_datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    preprocessing_function = preprocess_input)
test_datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    horizontal_flip=False,
    preprocessing_function = preprocess_input)

In [7]:
# from keras.applications.xception import preprocess_input
# x_train_array_pre = preprocess_input(x_train_array)
# x_test_array_pre = preprocess_input(x_test_array)
train_datagen.fit(x_train_array)
test_datagen.fit(x_test_array)

In [8]:
train_data = train_datagen.flow(x_train_array, y_train, batch_size=1)
test_data = test_datagen.flow(x_test_array, y_test)

In [9]:
from keras.callbacks import ModelCheckpoint
batch_size = 8
nb_epoch = 600
filepath="weights.best.h5"
checkpoint= ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list= [checkpoint]
my_model = model.fit_generator(train_data,
                    steps_per_epoch=x_train_array.shape[0] // batch_size,
                    validation_data=test_data,
                    epochs=nb_epoch,
                    callbacks=callbacks_list)

Epoch 1/600
122/123 [============================>.] - ETA: 0s - loss: 2.1286 - acc: 0.1148
Epoch 00001: val_acc improved from -inf to 0.14657, saving model to weights.best.h5
123/123 [==============================] - 24s 192ms/step - loss: 2.1356 - acc: 0.1138 - val_loss: 2.7981 - val_acc: 0.1466
Epoch 2/600
122/123 [============================>.] - ETA: 0s - loss: 2.0370 - acc: 0.1475
Epoch 00002: val_acc improved from 0.14657 to 0.14894, saving model to weights.best.h5
123/123 [==============================] - 12s 96ms/step - loss: 2.0357 - acc: 0.1463 - val_loss: 2.9282 - val_acc: 0.1489
Epoch 3/600
122/123 [============================>.] - ETA: 0s - loss: 2.0873 - acc: 0.1721
Epoch 00003: val_acc improved from 0.14894 to 0.15130, saving model to weights.best.h5
123/123 [==============================] - 12s 95ms/step - loss: 2.0831 - acc: 0.1707 - val_loss: 2.2101 - val_acc: 0.1513
Epoch 4/600
122/123 [============================>.] - ETA: 0s - loss: 2.0322 - acc: 0.1311
Epoc

Epoch 33/600
122/123 [============================>.] - ETA: 0s - loss: 2.0389 - acc: 0.1148
Epoch 00033: val_acc did not improve
123/123 [==============================] - 10s 80ms/step - loss: 2.0370 - acc: 0.1138 - val_loss: 1.9982 - val_acc: 0.1442
Epoch 34/600
122/123 [============================>.] - ETA: 0s - loss: 1.9665 - acc: 0.1885
Epoch 00034: val_acc did not improve
123/123 [==============================] - 10s 80ms/step - loss: 1.9642 - acc: 0.1870 - val_loss: 2.0020 - val_acc: 0.1489
Epoch 35/600
122/123 [============================>.] - ETA: 0s - loss: 1.9512 - acc: 0.1803
Epoch 00035: val_acc did not improve
123/123 [==============================] - 10s 80ms/step - loss: 1.9521 - acc: 0.1789 - val_loss: 1.9957 - val_acc: 0.1300
Epoch 36/600
122/123 [============================>.] - ETA: 0s - loss: 1.8911 - acc: 0.2213
Epoch 00036: val_acc did not improve
123/123 [==============================] - 10s 80ms/step - loss: 1.8885 - acc: 0.2195 - val_loss: 1.9705 - val_

Epoch 64/600
122/123 [============================>.] - ETA: 0s - loss: 1.0650 - acc: 0.5492
Epoch 00064: val_acc did not improve
123/123 [==============================] - 9s 76ms/step - loss: 1.0643 - acc: 0.5447 - val_loss: 1.7000 - val_acc: 0.2908
Epoch 65/600
122/123 [============================>.] - ETA: 0s - loss: 1.1177 - acc: 0.5820
Epoch 00065: val_acc did not improve
123/123 [==============================] - 10s 81ms/step - loss: 1.1240 - acc: 0.5772 - val_loss: 1.6935 - val_acc: 0.3050
Epoch 66/600
122/123 [============================>.] - ETA: 0s - loss: 0.9975 - acc: 0.5902
Epoch 00066: val_acc did not improve
123/123 [==============================] - 10s 80ms/step - loss: 1.0058 - acc: 0.5854 - val_loss: 1.7734 - val_acc: 0.3073
Epoch 67/600
122/123 [============================>.] - ETA: 0s - loss: 1.1440 - acc: 0.5164
Epoch 00067: val_acc improved from 0.31206 to 0.32861, saving model to weights.best.h5
123/123 [==============================] - 11s 92ms/step - los

Epoch 96/600
122/123 [============================>.] - ETA: 0s - loss: 0.8541 - acc: 0.6148
Epoch 00096: val_acc did not improve
123/123 [==============================] - 10s 80ms/step - loss: 0.8620 - acc: 0.6098 - val_loss: 1.6458 - val_acc: 0.3617
Epoch 97/600
122/123 [============================>.] - ETA: 0s - loss: 0.8408 - acc: 0.6721
Epoch 00097: val_acc did not improve
123/123 [==============================] - 10s 81ms/step - loss: 0.8362 - acc: 0.6748 - val_loss: 1.7046 - val_acc: 0.3712
Epoch 98/600
122/123 [============================>.] - ETA: 0s - loss: 0.9194 - acc: 0.5738
Epoch 00098: val_acc did not improve
123/123 [==============================] - 10s 80ms/step - loss: 0.9196 - acc: 0.5691 - val_loss: 1.7558 - val_acc: 0.3664
Epoch 99/600
122/123 [============================>.] - ETA: 0s - loss: 0.8852 - acc: 0.6311
Epoch 00099: val_acc improved from 0.38061 to 0.39716, saving model to weights.best.h5
123/123 [==============================] - 12s 96ms/step - lo

Epoch 127/600
122/123 [============================>.] - ETA: 0s - loss: 0.5753 - acc: 0.7869
Epoch 00127: val_acc did not improve
123/123 [==============================] - 10s 81ms/step - loss: 0.5829 - acc: 0.7805 - val_loss: 1.6749 - val_acc: 0.4232
Epoch 128/600
122/123 [============================>.] - ETA: 0s - loss: 0.6430 - acc: 0.7623
Epoch 00128: val_acc did not improve
123/123 [==============================] - 10s 80ms/step - loss: 0.6381 - acc: 0.7642 - val_loss: 1.6925 - val_acc: 0.4350
Epoch 129/600
122/123 [============================>.] - ETA: 0s - loss: 0.6633 - acc: 0.6639
Epoch 00129: val_acc did not improve
123/123 [==============================] - 10s 80ms/step - loss: 0.6620 - acc: 0.6667 - val_loss: 1.7393 - val_acc: 0.4374
Epoch 130/600
122/123 [============================>.] - ETA: 0s - loss: 0.5417 - acc: 0.7295
Epoch 00130: val_acc did not improve
123/123 [==============================] - 10s 80ms/step - loss: 0.5430 - acc: 0.7317 - val_loss: 1.6661 - 

...

122/123 [============================>.] - ETA: 0s - loss: 0.1884 - acc: 0.9180
Epoch 00319: val_acc did not improve
123/123 [==============================] - 10s 80ms/step - loss: 0.1871 - acc: 0.9187 - val_loss: 2.1041 - val_acc: 0.4799
Epoch 320/600
122/123 [============================>.] - ETA: 0s - loss: 0.1786 - acc: 0.9426
Epoch 00320: val_acc did not improve
123/123 [==============================] - 10s 80ms/step - loss: 0.1771 - acc: 0.9431 - val_loss: 2.2879 - val_acc: 0.4610
Epoch 321/600
122/123 [============================>.] - ETA: 0s - loss: 0.2726 - acc: 0.9016
Epoch 00321: val_acc did not improve
123/123 [==============================] - 10s 80ms/step - loss: 0.2705 - acc: 0.9024 - val_loss: 2.5443 - val_acc: 0.4657
Epoch 322/600
122/123 [============================>.] - ETA: 0s - loss: 0.2394 - acc: 0.9262
Epoch 00322: val_acc did not improve
123/123 [==============================] - 10s 80ms/step - loss: 0.2374 - acc: 0.9268 - val_loss: 2.8747 - val_acc: 0.465

122/123 [============================>.] - ETA: 0s - loss: 0.0469 - acc: 0.9918
Epoch 00383: val_acc did not improve
123/123 [==============================] - 9s 76ms/step - loss: 0.0466 - acc: 0.9919 - val_loss: 2.2594 - val_acc: 0.4870
Epoch 384/600
122/123 [============================>.] - ETA: 0s - loss: 0.1660 - acc: 0.9426
Epoch 00384: val_acc did not improve
123/123 [==============================] - 9s 76ms/step - loss: 0.1646 - acc: 0.9431 - val_loss: 2.6788 - val_acc: 0.4563
Epoch 385/600
122/123 [============================>.] - ETA: 0s - loss: 0.1010 - acc: 0.9754
Epoch 00385: val_acc did not improve
123/123 [==============================] - 9s 75ms/step - loss: 0.1002 - acc: 0.9756 - val_loss: 2.2220 - val_acc: 0.4728
Epoch 386/600
122/123 [============================>.] - ETA: 0s - loss: 0.2987 - acc: 0.9262
Epoch 00386: val_acc did not improve
123/123 [==============================] - 9s 75ms/step - loss: 0.2978 - acc: 0.9268 - val_loss: 2.4445 - val_acc: 0.4515
Ep

122/123 [============================>.] - ETA: 0s - loss: 0.1430 - acc: 0.9508
Epoch 00447: val_acc did not improve
123/123 [==============================] - 10s 81ms/step - loss: 0.1418 - acc: 0.9512 - val_loss: 3.6453 - val_acc: 0.4586
Epoch 448/600
122/123 [============================>.] - ETA: 0s - loss: 0.0893 - acc: 0.9590
Epoch 00448: val_acc did not improve
123/123 [==============================] - 10s 80ms/step - loss: 0.0886 - acc: 0.9593 - val_loss: 3.4817 - val_acc: 0.4799
Epoch 449/600
122/123 [============================>.] - ETA: 0s - loss: 0.1551 - acc: 0.9508
Epoch 00449: val_acc did not improve
123/123 [==============================] - 10s 81ms/step - loss: 0.1538 - acc: 0.9512 - val_loss: 3.3942 - val_acc: 0.5154
Epoch 450/600
122/123 [============================>.] - ETA: 0s - loss: 0.1327 - acc: 0.9426
Epoch 00450: val_acc improved from 0.52719 to 0.53191, saving model to weights.best.h5
123/123 [==============================] - 12s 97ms/step - loss: 0.1317

Epoch 479/600
122/123 [============================>.] - ETA: 0s - loss: 0.1068 - acc: 0.9672
Epoch 00479: val_acc did not improve
123/123 [==============================] - 10s 81ms/step - loss: 0.1061 - acc: 0.9675 - val_loss: 3.2934 - val_acc: 0.4492
Epoch 480/600
122/123 [============================>.] - ETA: 0s - loss: 0.1944 - acc: 0.9180
Epoch 00480: val_acc did not improve
123/123 [==============================] - 10s 81ms/step - loss: 0.1929 - acc: 0.9187 - val_loss: 2.5995 - val_acc: 0.5035
Epoch 481/600
122/123 [============================>.] - ETA: 0s - loss: 0.1993 - acc: 0.9180
Epoch 00481: val_acc did not improve
123/123 [==============================] - 10s 81ms/step - loss: 0.1977 - acc: 0.9187 - val_loss: 2.3981 - val_acc: 0.5225
Epoch 482/600
122/123 [============================>.] - ETA: 0s - loss: 0.0307 - acc: 1.0000
Epoch 00482: val_acc did not improve
123/123 [==============================] - 10s 80ms/step - loss: 0.0305 - acc: 1.0000 - val_loss: 2.4147 - 

122/123 [============================>.] - ETA: 0s - loss: 0.0571 - acc: 0.9836
Epoch 00543: val_acc did not improve
123/123 [==============================] - 10s 81ms/step - loss: 0.0567 - acc: 0.9837 - val_loss: 3.7686 - val_acc: 0.4255
Epoch 544/600
122/123 [============================>.] - ETA: 0s - loss: 0.1204 - acc: 0.9590
Epoch 00544: val_acc did not improve
123/123 [==============================] - 10s 81ms/step - loss: 0.1195 - acc: 0.9593 - val_loss: 4.0292 - val_acc: 0.4350
Epoch 545/600
122/123 [============================>.] - ETA: 0s - loss: 0.0761 - acc: 0.9836
Epoch 00545: val_acc did not improve
123/123 [==============================] - 10s 81ms/step - loss: 0.0755 - acc: 0.9837 - val_loss: 3.7928 - val_acc: 0.4444
Epoch 546/600
122/123 [============================>.] - ETA: 0s - loss: 0.0588 - acc: 0.9836
Epoch 00546: val_acc did not improve
123/123 [==============================] - 10s 81ms/step - loss: 0.0596 - acc: 0.9837 - val_loss: 3.9984 - val_acc: 0.413

In [ ]:
import h5py
model.save_weights('weights_2.h5')

In [10]:
model.save('model_2.h5')

In [ ]:
print(my_model)

In [ ]:
from keras.applications.xception import Xception
from keras.layers import *
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.core import Dropout
def res_net():
    input = Input(shape=[100, 100, 3])
    nub_class = 7
    x = Conv2D(64,(3,3),activation='relu', padding='same', name='block1_conv1')(input)
    x = Conv2D(64,(3,3),activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    output = Dense(nub_class,activation='softmax')(x)
    
    model = Model(input, output)
    
    return model